In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.stats import norm, expon
import pandas_datareader.data as web
import math
import datetime as dt

In [2]:
def data_preprocess(price,window):
    
    log_return = np.log(price/price.shift(1))
    log_return_sq = log_return**2
    ma = log_return.rolling(window*252).mean() ## daily log return moving average
    #variance = log_return_sq.rolling(window*252).mean() - ma**2 #daily log return variance
    #sd = variance**0.5  #daily log return variance
    sd = log_return.rolling(window*252).std()
    sigma = sd/((1/252)**0.5)  
    mu = ma/(1/252) + (sigma**2)/2 
    
    return sigma,mu,log_return


N = norm.cdf

def BS_CALL(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * N(d1) - K * np.exp(-r*T)* N(d2)

def BS_PUT(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma* np.sqrt(T)
    return K*np.exp(-r*T)*N(-d2) - S*N(-d1)

In [101]:
spy = web.DataReader(name='SPY', data_source='yahoo',start='2000-10-05', end = '2020-02-04')
sigma,mu,log_return = data_preprocess(spy['Close'],10)
s = sigma[-1]
m = mu[-1]

In [104]:
price = 378
#s = 0.420010
#m = -0.387797
print(s,m)
t = 7
p = 0.90
n = 1000000
strike = price * np.exp(s* ((t/252)**0.5) * norm.ppf(1 - p) + (m - s**2/2)*(t/252))
print(p,strike)
simulated_p = price*np.exp(np.random.normal(0, (s**2 * t/252)**0.5 ,n) + (m - s**2/2)*t/252)
simulated_p.sort()
print(simulated_p[int(n*(1-p))])

0.14743788364743796 0.12284626830247246
0.9 367.42276544274216
367.40375840434893


In [108]:
## fixed profit
iv = 0.15
put = BS_PUT(378,strike,t/252,0.05,iv)
print('put_price',put)
payoff = [min(s-strike, 0)+put for s in simulated_p]
print('mean payoff',np.mean(payoff))

put_price 0.5322424335029439
mean payoff 0.10911364804707668
